<a href="https://colab.research.google.com/github/tul17ii/Mini-Fitness-Function-Project/blob/main/Mini_Fitness_Function_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

# Step 1: Define possible values for genes
crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Step 2: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    return random.uniform(0, 50)

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 3: Generate random chromosome with added soil nutrients, farm size, and farm type
def generate_random_chromosome():
    # Limit fertilizer selection based on farm size
    farm_size = random.choice(farm_sizes)
    if farm_size == "Small":
        available_fertilizers = ["Organic", "Nitrogen"]
    elif farm_size == "Medium":
        available_fertilizers = fertilizers
    else:  # Large farms have all fertilizers
        available_fertilizers = fertilizers

    fertilizer = random.choice(available_fertilizers)

    # Limit pesticide selection based on farm type
    farm_type = random.choice(farm_types)
    if farm_type == "Organic":
        available_pesticides = ["None"]
    else:  # Conventional farms can use any pesticide
        available_pesticides = pesticides

    pesticide = random.choice(available_pesticides)

    # Limit soil nutrient selection based on farm size
    soil_nutrient = random.choice(soil_nutrients)

    return [
        random.choice(crop_types),
        random.choice(planting_times),
        fertilizer,
        pesticide,
        soil_nutrient,
        farm_size,
        farm_type
    ]

# Step 4: Initialize population
population_size = 10
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 5: Parent selection using fitness-based roulette selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)

    # Normalize fitness values to sum to 1
    normalized_fitness = [fitness / total_fitness for fitness in fitness_values]

    # Select two parents based on fitness probabilities
    parent1 = random.choices(population, weights=normalized_fitness, k=1)[0]
    parent2 = random.choices(population, weights=normalized_fitness, k=1)[0]

    return parent1, parent2

# Step 6: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 7: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        chromosome[mutation_point] = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        chromosome[mutation_point] = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        chromosome[mutation_point] = random.choice(farm_sizes)
    else:  # Farm type mutation
        chromosome[mutation_point] = random.choice(farm_types)

# Step 8: Mutation with temperature
def mutate_with_temperature(chromosome, temperature):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        new_value = random.choice(crop_types)
    elif mutation_point == 1:
        new_value = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        new_value = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        new_value = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        new_value = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        new_value = random.choice(farm_sizes)
    else:  # Farm type mutation
        new_value = random.choice(farm_types)

    # Simulated annealing-inspired acceptance
    original_fitness = combined_fitness(chromosome)
    chromosome[mutation_point] = new_value
    new_fitness = combined_fitness(chromosome)

    if new_fitness < original_fitness:
        acceptance_probability = random.uniform(0, 1)
        if acceptance_probability > (1 / (1 + temperature)):  # Revert mutation with a probability
            chromosome[mutation_point] = new_value

    return chromosome

# Step 9: User input for fitness function selection
print("Select a fitness function:")
print("1. Crop Yield Fitness")
print("2. Environmental Impact Fitness")
print("3. Combined Fitness")

user_choice = int(input("Enter 1, 2, or 3: "))

if user_choice == 1:
    fitness_function = crop_yield_fitness
elif user_choice == 2:
    fitness_function = environmental_impact_fitness
elif user_choice == 3:
    fitness_function = combined_fitness
else:
    print("Invalid choice! Defaulting to Crop Yield Fitness.")
    fitness_function = crop_yield_fitness

# Step 10: Genetic Algorithm with Temperature and Cooling Rate
def genetic_algorithm_with_temperature(fitness_function, generations=10):
    population = initialize_population(population_size)
    T = 100  # Initial temperature
    alpha = 0.95  # Cooling rate

    for generation in range(generations):
        # Sort population by fitness
        population = sorted(population, key=fitness_function, reverse=True)

        # Print the best solution in this generation
        best_solution = population[0]
        print(f"Generation {generation + 1} - Best Solution: {best_solution}, Fitness: {fitness_function(best_solution)}, Temperature: {T}")

        # Create new generation
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = select_parents(population, fitness_function)
            child1, child2 = crossover(parent1, parent2)

            # Mutate the children with temperature influence
            child1 = mutate_with_temperature(child1, T)
            child2 = mutate_with_temperature(child2, T)

            next_generation.append(child1)
            next_generation.append(child2)

        population = next_generation[:population_size]
        T *= alpha  # Cool down the temperature

    best_solution = max(population, key=fitness_function)
    return best_solution, fitness_function(best_solution)

# Step 11: Run Genetic Algorithm with the selected fitness function
best_ga_solution, best_ga_fitness = genetic_algorithm_with_temperature(fitness_function)
print(f"\nBest solution using Genetic Algorithm: {best_ga_solution} with Fitness: {best_ga_fitness}")

Select a fitness function:
1. Crop Yield Fitness
2. Environmental Impact Fitness
3. Combined Fitness
Enter 1, 2, or 3: 1
Generation 1 - Best Solution: ['Corn', 'Winter', 'Organic', 'None', 'Medium', 'Small', 'Conventional'], Fitness: 17.175434361294574, Temperature: 100
Generation 2 - Best Solution: ['Rice', 'Winter', 'Organic', 'Herbicide', 'Low', 'Medium', 'Conventional'], Fitness: 57.82047987701735, Temperature: 95.0
Generation 3 - Best Solution: ['Rice', 'Fall', 'Organic', 'Insecticide', 'Low', 'Small', 'Conventional'], Fitness: 79.56385145387455, Temperature: 90.25
Generation 4 - Best Solution: ['Wheat', 'Spring', 'Nitrogen', 'None', 'High', 'Medium', 'Conventional'], Fitness: 54.586817035849364, Temperature: 85.7375
Generation 5 - Best Solution: ['Soybean', 'Spring', 'Nitrogen', 'Fungicide', 'Low', 'Medium', 'Conventional'], Fitness: 43.12902045226865, Temperature: 81.45062499999999
Generation 6 - Best Solution: ['Wheat', 'Spring', 'Organic', 'None', 'Medium', 'Medium', 'Organic'